In [33]:
import json
import requests
import pandas as pd
import time
from tqdm import tqdm

In [4]:
with open('keys.json') as f:
  keysRAW = f.read()
keys = json.loads(keysRAW)

In [5]:
api_url = "https://na1.api.riotgames.com"
grandmasterLeagueEnd = "/tft/league/v1/grandmaster"
response = requests.get(f"{api_url}{grandmasterLeagueEnd}", headers={"X-Riot-Token": keys["private"]})
response.status_code

200

In [7]:
GRANDMASTER = response.json()
GRANDMASTER

{'tier': 'GRANDMASTER',
 'leagueId': '19edec7a-a457-36a3-897d-a2f68b4c42a4',
 'queue': 'RANKED_TFT',
 'name': "Irelia's Weaponmasters",
 'entries': [{'summonerId': 'F2gVk8Fjwfq3eti_GjQSFoFlxY8ynFHKCKK9BuYJW8neQsCu',
   'summonerName': 'Zhu 8 Jie',
   'leaguePoints': 480,
   'rank': 'I',
   'wins': 225,
   'losses': 170,
   'veteran': False,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': 'BcvaGbyOYPSthyxn8b8Dkvso4lyEBQpd4mTpC9xc6rGXoa8',
   'summonerName': 'ConstantlyMaldin',
   'leaguePoints': 516,
   'rank': 'I',
   'wins': 270,
   'losses': 236,
   'veteran': False,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': '9l_pX1eIB9NDTEHW2jK4O8cQZbtL0ws2ronirkAJq9cIaZ8',
   'summonerName': 'BYongski79',
   'leaguePoints': 536,
   'rank': 'I',
   'wins': 346,
   'losses': 315,
   'veteran': False,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': 'I6yN0Rz6T2cURyeQ5z4p25EH7V8PmsC

In [8]:
summonerIDs = []
for summoner in GRANDMASTER["entries"]:
  summonerIDs.append(summoner["summonerId"])

summonerIDs

['F2gVk8Fjwfq3eti_GjQSFoFlxY8ynFHKCKK9BuYJW8neQsCu',
 'BcvaGbyOYPSthyxn8b8Dkvso4lyEBQpd4mTpC9xc6rGXoa8',
 '9l_pX1eIB9NDTEHW2jK4O8cQZbtL0ws2ronirkAJq9cIaZ8',
 'I6yN0Rz6T2cURyeQ5z4p25EH7V8PmsC0iENhZ6s7Q0rnKUg',
 'kq11dWnbA2AozBDGw4uDWCaS98dZV4upWegIX4cnpFE2XTdNLsF4_E9zzg',
 '8tnFZSlJbJ8X8ZjBRA0XgX6vsm7H0LcQeYUG5oJiwlpAr1U-',
 'Fk6IsmbxOfI7LDrj3LXDcKPtNh7jR7nCJniaoIGxZ6-9Q0I',
 'b-ZogkBdpuKV3pmpww_KOwKmbzNXnFyzFNzp2pvLhOBjrsM',
 'XFGz1Yt5ysE2ADrjs4Sh-0q8L_e4lmVesvIzEuaP2qFJN2M',
 'wZr8E_rXNTwLEk4jtQTHLWrJEfiKouZqe0MIyQBY7Ydvj2A',
 'vyvMrjR6s5A1EeiYJmyAJEu3VjZhgvNJ1LYP0Ci1kQ2ijMs',
 'wlNE8lZh1OjcTHZc7MoF6AG-0mMSQIV3nwN3jNE0r7asABs',
 'm9XSthAAfK-1pV3p_1kb3nHpgUQ39lCxPeIW_vnB6qIdFJQ',
 'bdwAHBX7xMComQJ1aojM6SdvmpSzDZX9PfvKPai-mJ0L87A',
 '-ZQPK5O0DkILI92CWLxsr8C4DGL4bRvvFKvgziIsYdKTWGk',
 'naOx4QCEgz2hItknktayHHIpCDCE7hr9wOMjSQFuAvuKtjQ',
 '0dwWwckfVagdfH1KMO3r06dqhbgT-MR24rFt8JWWG5M9x0E',
 'ohFGLJHtdPhInx3vUo9NnJ6Y_8zuxmBEAeOrMB_FZ5qca6lm',
 '-5KTDT0HM68U0KItRjgP0b4_-tRNzDBO6brnwtJFPaJwMHo'

In [34]:
def gatherSummonerData (summonerID):
  calls = 0

  summonerEnd = "/tft/summoner/v1/summoners/"
  puuidsResponse = requests.get(f"{api_url}{summonerEnd}{summonerID}", headers={"X-Riot-Token": keys["private"]})
  calls += 1
  puuid = puuidsResponse.json()["puuid"]

  api_url_region_america = "https://americas.api.riotgames.com"
  matchPUUIDEnd = f"/tft/match/v1/matches/by-puuid/{puuid}/ids"
  start = 0
  count = 20
  matchesResponse = requests.get(f"{api_url_region_america}{matchPUUIDEnd}?start={start}&count={count}", headers={"X-Riot-Token": keys["private"]})
  calls += 1

  matches = matchesResponse.json()

  summonerMatchData = []
  for match in tqdm(matches):
    matchEND = "/tft/match/v1/matches/"
    matchDataResponse = requests.get(f"{api_url_region_america}{matchEND}{match}", headers={"X-Riot-Token": keys["private"]})
    calls += 1
    time.sleep(1)

    matchData = {}
    for participant in matchDataResponse.json()["info"]["participants"]:
      matchData[participant["puuid"]] = {}
      matchData[participant["puuid"]]["placement"] = participant["placement"]
      matchData[participant["puuid"]]["traits"] = []
      matchData[participant["puuid"]]["units"] = []
      matchData[participant["puuid"]]["augments"] = []
      for trait in participant["traits"]:
        matchData[participant["puuid"]]["traits"].append(trait["name"])
      for unit in participant["units"]:
         matchData[participant["puuid"]]["units"].append(unit["character_id"])
      for augment in participant["augments"]:
        matchData[participant["puuid"]]["augments"].append(augment)

    summonerMatchData.append(matchData)

  return summonerMatchData

In [35]:
grandmasterData = {}
grandmasterData[summonerIDs[0]] = gatherSummonerData(summonerIDs[0])

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:41<00:00,  2.05s/it]


In [36]:
grandmasterData

{'F2gVk8Fjwfq3eti_GjQSFoFlxY8ynFHKCKK9BuYJW8neQsCu': [{'w224qmiXE7VTLx0MDMn3mlxzZY-w2ogMVttCXC458OX2Ok7HwU1MsH13RxpLtxY2-xRqcOD2EWhJBQ': {'placement': 5,
    'traits': ['Set10_Breakout',
     'Set10_Edgelord',
     'Set10_Executioner',
     'Set10_Fighter',
     'Set10_Guardian',
     'Set10_KDA',
     'Set10_Pentakill',
     'Set10_Sentinel',
     'Set10_Superfan'],
    'units': ['TFT10_Lillia',
     'TFT10_Gnar',
     'TFT10_Kayle',
     'TFT10_Neeko',
     'TFT10_Mordekaiser',
     'TFT10_Viego',
     'TFT10_Karthus',
     'TFT10_Akali',
     'TFT10_Yorick'],
    'augments': ['TFT9_Augment_HighEndSector',
     'TFT10_Augment_HeroicGrabBag',
     'TFT9_Augment_GiftsFromTheFallen']},
   'efCjynOk0hn5aAz6ULL7STm-IiuJoKzdr6NftJXnwa6FBsAcEuZPXqI9JL8n-komcqzklAW4EY51wQ': {'placement': 7,
    'traits': ['Set10_Breakout',
     'Set10_Dazzler',
     'Set10_Deadeye',
     'Set10_Executioner',
     'Set10_Guardian',
     'Set10_Hyperpop',
     'Set10_KDA',
     'Set10_Sentinel',
     'Set10_Sp

In [37]:
with open("grandmasterData.json", "w") as outfile:
    json.dump(grandmasterData, outfile)